<a href="https://colab.research.google.com/github/tony-pitchblack/finrl-dt/blob/custom-backtesting/finrl_dt_replicate_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install stable-baselines3
!pip install finrl
!pip install alpaca_trade_api
!pip install exchange_calendars
!pip install stockstats
!pip install wrds

# Imports

In [ ]:
import os
import wandb
import pandas as pd

In [ ]:
os.environ["WANDB_API_KEY"] = "aee284a72205e2d6787bd3ce266c5b9aefefa42c"

PROJECT = 'finrl-dt-replicate'
ENTITY = "overfit1010"

# Load models and dataset

In [ ]:
# !rm -rf ./*

RUN_ID = '4resq86d'

In [ ]:
#@title Download artifacts

# Initialize the W&B API
api = wandb.Api()

# Retrieve the run
run = api.run(f"{ENTITY}/{PROJECT}/{RUN_ID}")

# Iterate over the artifacts used or logged by the run
for artifact in run.logged_artifacts():
    artifact.download(f'./{artifact.type}')

True


wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   4 of 4 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   6 of 6 files downloaded.  
wandb:   3 of 3 files downloaded.  
wandb:   8 of 8 files downloaded.  
wandb:   4 of 4 files downloaded.  
wandb:   10 of 10 files downloaded.  
wandb:   5 of 5 files downloaded.  


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ./results/

# Eval models

In [ ]:
#@title imports

import sys
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

from finrl.config import INDICATORS, TRAINED_MODEL_DIR

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#@title Load data

train = pd.read_csv('./dataset/train_data.csv')
trade = pd.read_csv('./dataset/test_data.csv')

train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

In [ ]:
#@title Define metric functions

def calculate_mdd(asset_values):
    """
    Calculate the Maximum Drawdown (MDD) of a portfolio.
    """
    running_max = asset_values.cummax()
    drawdown = (asset_values - running_max) / running_max
    mdd = drawdown.min() * 100  # Convert to percentage
    return mdd

def calculate_sharpe_ratio(asset_values, risk_free_rate=0.0):
    """
    Calculate the Sharpe Ratio of a portfolio.
    """
    # Calculate daily returns
    returns = asset_values.pct_change().dropna()
    excess_returns = returns - risk_free_rate / 252  # Assuming 252 trading days
    if excess_returns.std() == 0:
        return 0.0
    sharpe_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)  # Annualized
    return sharpe_ratio

def calculate_annualized_return(asset_values):
    """
    Calculate the annualized return of a portfolio.
    """
    # Assume `asset_values` is indexed by date or trading day
    total_return = (asset_values.iloc[-1] / asset_values.iloc[0] - 1) * 100
    num_days = (asset_values.index[-1] - asset_values.index[0]).days
    annualized_return = (1 + total_return) ** (365 / num_days) - 1
    return annualized_return

In [ ]:
wandb.init(entity=ENTITY, project=PROJECT, id=RUN_ID, resume='must')
config = wandb.run.config

## Eval default

In [ ]:
#@title EVAL default
# comparison under expert agent

os.makedirs('./checkpoints', exist_ok=True)

algorithms = [
    'a2c',
    'ddpg',
    'ppo',
    'td3',
    'sac'
]

# Define a mapping for better legend labels
label_mapping = {
    # 'DT_LoRA_GPT2': 'DT-LoRA-GPT2',
    # 'DT_LoRA_Random_Weight_GPT2': 'DT-LoRA-Random-GPT2',
    # 'CQL': 'Conservative Q-Learning',
    # 'IQL': 'Implicit Q-Learning',
    # 'BC': 'Behavior Cloning',
    'A2C': 'A2C',
    'DDPG': 'DDPG',
    'PPO': 'PPO',
    'TD3': 'TD3',
    'SAC': 'SAC',
    'DJIA': 'Dow Jones Index'
}

for current_algo in algorithms:
    # Reset all algorithms to False
    if_using_a2c = False
    if_using_ddpg = False
    if_using_ppo = False
    if_using_td3 = False
    if_using_sac = False

    # Set the current algorithm to True
    if current_algo == 'a2c':
        if_using_a2c = True
    elif current_algo == 'ddpg':
        if_using_ddpg = True
    elif current_algo == 'ppo':
        if_using_ppo = True
    elif current_algo == 'td3':
        if_using_td3 = True
    elif current_algo == 'sac':
        if_using_sac = True

    # Reset algos_included for each iteration
    algos_included = ''

    print(f"Running with {current_algo.upper()} set to True")

    # Load trained models
    trained_a2c = A2C.load(os.path.join(TRAINED_MODEL_DIR, "agent_a2c")) if if_using_a2c else None
    trained_ddpg = DDPG.load(os.path.join(TRAINED_MODEL_DIR, "agent_ddpg")) if if_using_ddpg else None
    trained_ppo = PPO.load(os.path.join(TRAINED_MODEL_DIR, "agent_ppo")) if if_using_ppo else None
    trained_td3 = TD3.load(os.path.join(TRAINED_MODEL_DIR, "agent_td3")) if if_using_td3 else None
    trained_sac = SAC.load(os.path.join(TRAINED_MODEL_DIR, "agent_sac")) if if_using_sac else None

    # Define environment parameters
    stock_dimension = len(trade.tic.unique())
    state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

    num_stock_shares = [0] * stock_dimension

    env_kwargs = {
        "hmax": 100,
        "initial_amount": config['initial_amount'],
        "num_stock_shares": num_stock_shares,
        "buy_cost_pct": config['cost_pct'],
        "sell_cost_pct": config['cost_pct'],
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": INDICATORS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4
    }

    # Initialize trading environment
    e_trade_gym = StockTradingEnv(
        df=trade,
        turbulence_threshold=70, risk_indicator_col='vix',
        **env_kwargs
    )

    env_trade, obs_trade = e_trade_gym.get_sb_env()
    # Predict using trained models
    df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
        model=trained_a2c,
        environment=e_trade_gym) if if_using_a2c else (None, None)

    df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
        model=trained_ddpg,
        environment=e_trade_gym) if if_using_ddpg else (None, None)

    df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
        model=trained_td3,
        environment=e_trade_gym) if if_using_td3 else (None, None)

    df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
        model=trained_sac, environment=e_trade_gym) if if_using_sac else (None, None)

    df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
        model=trained_ppo, environment=e_trade_gym) if if_using_ppo else (None, None)

    # Set indices for result merging
    df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0]) if if_using_a2c else None
    df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0]) if if_using_ddpg else None
    df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0]) if if_using_ppo else None
    df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0]) if if_using_td3 else None
    df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0]) if if_using_sac else None

    # Merge results
    result = pd.DataFrame()
    if if_using_a2c:
        algos_included += '_a2c'
        df_result_a2c.columns = ['A2C_' + col for col in df_result_a2c.columns]
        result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
    if if_using_ddpg:
        algos_included += '_ddpg'
        df_result_ddpg.columns = ['DDPG_' + col for col in df_result_ddpg.columns]
        result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
    if if_using_td3:
        algos_included += '_td3'
        df_result_td3.columns = ['TD3_' + col for col in df_result_td3.columns]
        result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
    if if_using_sac:
        algos_included += '_sac'
        df_result_sac.columns = ['SAC_' + col for col in df_result_sac.columns]
        result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
    if if_using_ppo:
        algos_included += '_ppo'
        df_result_ppo.columns = ['PPO_' + col for col in df_result_ppo.columns]
        result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)

    # Function to load pickle files
    def load_pickle(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)

    # Get all directories under ./checkpoints/
    checkpoint_dirs = [d for d in os.listdir('./checkpoints') if os.path.isdir(os.path.join('./checkpoints', d))]

    # Process each checkpoint directory
    for dir_name in checkpoint_dirs:
        dir_path = os.path.join('./checkpoints', dir_name)
        pkl_files = [f for f in os.listdir(dir_path) if f.startswith('total_asset_value_change_test') and f.endswith('.pkl')]

        for pkl_file in pkl_files:
            file_path = os.path.join(dir_path, pkl_file)
            data = load_pickle(file_path)
            data = data[:335]  # Limit data to first 335 points

            # Validate data length
            if len(data) != len(result.index):
                print(f"Warning: Data length mismatch for {pkl_file} in {dir_name}. Expected {len(result.index)}, got {len(data)}. Skipping this file.")
                continue

            # Use directory name and pkl file name (without extension) as column name
            column_name = f'{dir_name}_{os.path.splitext(pkl_file)[0]}'

            df = pd.DataFrame(data, columns=[column_name])
            df.index = result.index  # Assuming the dates align with the existing result dataframe
            result = pd.merge(result, df, how='outer', left_index=True, right_index=True)

    # Create column names list with better formatting
    col_name = []
    if if_using_a2c: col_name.append('A2C')
    if if_using_ddpg: col_name.append('DDPG')
    if if_using_td3: col_name.append('TD3')
    if if_using_sac: col_name.append('SAC')
    if if_using_ppo: col_name.append('PPO')
    col_name.extend(result.columns[len(col_name):])  # Add all other column names
    result.columns = col_name

    # Define test period
    TEST_START_DATE = '2020-07-01'
    TEST_END_DATE = '2021-10-29'

    # Fetch DJIA data for the test period
    df_dji = YahooDownloader(start_date=TEST_START_DATE,
                             end_date=TEST_END_DATE,
                             ticker_list=['dji']).fetch_data()

    df_dji = df_dji[['date','close']]
    fst_day = df_dji['close'].iloc[0]
    dji = pd.DataFrame({
        'DJIA': df_dji['close'].div(fst_day).mul(1000000).values
    }, index=df_dji['date'])

    # Merge DJIA data using inner join to ensure alignment
    result = pd.merge(result, dji, how='inner', left_index=True, right_index=True).fillna(method='bfill')

    # Control variables
    include_ensemble = False  # Set to False to exclude ensemble experiments
    exclude_algo_experiments = False  # Set to True to exclude individual algo experiments based on if_using_... flags

    if include_ensemble:
        algos_included += '_ensemble'

    # Function to extract experiment name from directory name
    def extract_experiment_name(dir_name):
        # This regex matches everything up to the last underscore and number
        match = re.match(r'(.+)_\d+$', dir_name)
        if match:
            return match.group(1)
        return dir_name

    # Function to check if an experiment should be included
    def should_include_experiment(exp_name):
        if not include_ensemble and 'ensemble' in exp_name.lower():
            return False
        if exclude_algo_experiments:
            algo_flags = {
                'a2c': if_using_a2c,
                'ddpg': if_using_ddpg,
                'ppo': if_using_ppo,
                'td3': if_using_td3,
                'sac': if_using_sac
            }
            for algo, flag in algo_flags.items():
                if algo in exp_name.lower() and not flag:
                    return False
        return True

    # Group similar experiments
    experiment_groups = {}
    for column in result.columns:
        if column not in ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO', 'DJIA']:
            exp_name = extract_experiment_name(column.split('_total_asset_value_change_test')[0])
            if should_include_experiment(exp_name):
                if exp_name not in experiment_groups:
                    experiment_groups[exp_name] = []
                experiment_groups[exp_name].append(column)

    # Initialize a dictionary to store metrics for comparison
    metrics_dict = {
        'Method': [],
        'Cumulative Return Mean (%)': [],
        'Cumulative Return Std (%)': [],
        'MDD Mean (%)': [],
        'MDD Std (%)': [],
        'Sharpe Ratio Mean': [],
        'Sharpe Ratio Std': []
    }

    # Calculate metrics for each experiment group
    experiment_stats = {}
    for exp_name, columns in experiment_groups.items():
        exp_data = result[columns].dropna()

        if exp_data.empty:
            print(f"Warning: No valid data for experiment '{exp_name}'. Skipping metrics calculation.")
            continue

        # Cumulative Return: (Final - Initial) / Initial * 100 for each run
        cumulative_returns = (exp_data.iloc[-1] - exp_data.iloc[0]) / exp_data.iloc[0] * 100

        # Handle potential division by zero or invalid calculations
        cumulative_returns = cumulative_returns.replace([np.inf, -np.inf], np.nan).dropna()

        if cumulative_returns.empty:
            print(f"Warning: No valid cumulative returns for experiment '{exp_name}'. Skipping metrics calculation.")
            continue

        cumulative_return_mean = cumulative_returns.mean()
        cumulative_return_std = cumulative_returns.std()

        # MDD: Calculate MDD for each run
        mdd_values = []
        for col in columns:
            asset_values = result[col].dropna()
            if asset_values.empty:
                continue
            mdd_run = calculate_mdd(asset_values)
            mdd_values.append(mdd_run)

        if not mdd_values:
            print(f"Warning: No valid MDD values for experiment '{exp_name}'. Skipping MDD calculation.")
            mdd_mean = np.nan
            mdd_std = np.nan
        else:
            mdd_mean = np.mean(mdd_values)
            mdd_std = np.std(mdd_values)

        # Sharpe Ratio: Calculate Sharpe for each run
        sharpe_ratios = []
        for col in columns:
            asset_values = result[col].dropna()
            if asset_values.empty:
                continue
            sharpe_run = calculate_sharpe_ratio(asset_values)
            sharpe_ratios.append(sharpe_run)

        if not sharpe_ratios:
            print(f"Warning: No valid Sharpe Ratios for experiment '{exp_name}'. Skipping Sharpe Ratio calculation.")
            sharpe_mean = np.nan
            sharpe_std = np.nan
        else:
            sharpe_mean = np.mean(sharpe_ratios)
            sharpe_std = np.std(sharpe_ratios)

        # Append to metrics_dict with mapped label
        mapped_exp_name = label_mapping.get(exp_name, exp_name)
        metrics_dict['Method'].append(mapped_exp_name)
        metrics_dict['Cumulative Return Mean (%)'].append(cumulative_return_mean)
        metrics_dict['Cumulative Return Std (%)'].append(cumulative_return_std)
        metrics_dict['MDD Mean (%)'].append(mdd_mean)
        metrics_dict['MDD Std (%)'].append(mdd_std)
        metrics_dict['Sharpe Ratio Mean'].append(sharpe_mean)
        metrics_dict['Sharpe Ratio Std'].append(sharpe_std)

        # Store in experiment_stats for plotting
        experiment_stats[mapped_exp_name] = {'mean': exp_data.mean(axis=1), 'std': exp_data.std(axis=1)}

    # Calculate metrics for individual algorithms (A2C, DDPG, TD3, SAC, PPO)
    individual_algos = ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO']
    for algo in individual_algos:
        if algo in result.columns:
            # Check if this algorithm is already part of experiment_groups
            if label_mapping.get(algo, algo) in experiment_stats:
                print(f"Info: '{algo}' is already included in experiment groups. Skipping individual plotting to avoid duplication.")
                continue  # Skip to prevent duplicate plotting

            asset_values = result[algo].dropna()
            if asset_values.empty:
                print(f"Warning: No valid asset values for individual algorithm '{algo}'. Skipping metrics calculation.")
                continue
            # Cumulative Return
            cum_ret = (asset_values.iloc[-1] - asset_values.iloc[0]) / asset_values.iloc[0] * 100
            # Handle potential division by zero or invalid calculations
            if np.isinf(cum_ret) or np.isnan(cum_ret):
                cum_ret = np.nan
            # MDD
            mdd = calculate_mdd(asset_values)
            # Sharpe Ratio
            sharpe = calculate_sharpe_ratio(asset_values)
            # Append to metrics_dict with mapped label
            mapped_algo = label_mapping.get(algo, algo)
            metrics_dict['Method'].append(mapped_algo)
            metrics_dict['Cumulative Return Mean (%)'].append(cum_ret)
            metrics_dict['Cumulative Return Std (%)'].append(0.00)  # Single run, std is 0
            metrics_dict['MDD Mean (%)'].append(mdd)
            metrics_dict['MDD Std (%)'].append(0.00)  # Single run, std is 0
            metrics_dict['Sharpe Ratio Mean'].append(sharpe)
            metrics_dict['Sharpe Ratio Std'].append(0.00)  # Single run, std is 0

            # Store in experiment_stats for plotting
            experiment_stats[mapped_algo] = {'mean': asset_values, 'std': pd.Series([0]*len(asset_values), index=asset_values.index)}

    # Convert metrics_dict to DataFrame
    metrics_df = pd.DataFrame(metrics_dict)

    # Drop any rows with NaN metrics to ensure clean tables
    metrics_df = metrics_df.dropna(subset=['Cumulative Return Mean (%)', 'MDD Mean (%)', 'Sharpe Ratio Mean'])

    # Create summary DataFrame with formatted strings
    metrics_summary_df = metrics_df.copy()
    metrics_summary_df['Cumulative Return (%)'] = metrics_df['Cumulative Return Mean (%)'].round(2).astype(str) + " ± " + metrics_df['Cumulative Return Std (%)'].round(2).astype(str)
    metrics_summary_df['MDD (%)'] = metrics_df['MDD Mean (%)'].round(2).astype(str) + " ± " + metrics_df['MDD Std (%)'].round(2).astype(str)
    metrics_summary_df['Sharpe Ratio'] = metrics_df['Sharpe Ratio Mean'].round(2).astype(str) + " ± " + metrics_df['Sharpe Ratio Std'].round(2).astype(str)
    metrics_summary_df = metrics_summary_df[['Method', 'Cumulative Return (%)', 'MDD (%)', 'Sharpe Ratio']]

    # Print the comparison table
    print(f"\n=== Metrics Comparison for {current_algo.upper()} ===")
    print(metrics_summary_df.to_string(index=False))
    print("\n")

    # Create separate DataFrames for ranking
    ranking_cum_ret = metrics_df[['Method', 'Cumulative Return Mean (%)']].copy()
    ranking_cum_ret = ranking_cum_ret.sort_values(by='Cumulative Return Mean (%)', ascending=False)

    ranking_mdd = metrics_df[['Method', 'MDD Mean (%)']].copy()
    ranking_mdd = ranking_mdd.sort_values(by='MDD Mean (%)', ascending=True)  # Lower MDD is better

    ranking_sharpe = metrics_df[['Method', 'Sharpe Ratio Mean']].copy()
    ranking_sharpe = ranking_sharpe.sort_values(by='Sharpe Ratio Mean', ascending=False)

    # Print rankings
    print(f"=== Rankings for {current_algo.upper()} ===")

    print("\nCumulative Return (%):")
    for idx, row in ranking_cum_ret.iterrows():
        print(f"{row['Method']}: {row['Cumulative Return Mean (%)']:.2f}%")

    print("\nMaximum Drawdown (MDD %) [Lower is Better]:")
    for idx, row in ranking_mdd.iterrows():
        print(f"{row['Method']}: {row['MDD Mean (%)']:.2f}%")

    print("\nSharpe Ratio [Higher is Better]:")
    for idx, row in ranking_sharpe.iterrows():
        print(f"{row['Method']}: {row['Sharpe Ratio Mean']:.2f}")

    print("\n")

    # Debugging: Check if all means align with result.index
    for exp_name, stats in experiment_stats.items():
        mean_length = len(stats['mean'])
        result_length = len(result.index)
        if mean_length != result_length:
            print(f"Warning: Mean length for '{exp_name}' ({mean_length}) does not match result index length ({result_length}). Reindexing.")
            experiment_stats[exp_name]['mean'] = stats['mean'].reindex(result.index).fillna(method='ffill')
            experiment_stats[exp_name]['std'] = stats['std'].reindex(result.index).fillna(0)

    # Plotting section
    plt.figure(figsize=(16, 9))  # Increased figure size for better readability
    method_styles = {
        'CQL': {'color': '#1f77b4', 'linestyle': '-'},           # Blue solid
        'IQL': {'color': '#ff7f0e', 'linestyle': '--'},          # Orange dashed
        'BC': {'color': '#2ca02c', 'linestyle': '-.'},           # Green dash-dot
        'DT LoRA GPT2': {'color': '#d62728', 'linestyle': ':'},  # Red dotted
        'DT LoRA Random Weight GPT2': {'color': '#9467bd', 'linestyle': '-'},  # Purple solid
        'A2C': {'color': '#8c564b', 'linestyle': '--'},          # Brown dashed
        'DDPG': {'color': '#e377c2', 'linestyle': '-'},          # Pink solid
        'PPO': {'color': '#7f7f7f', 'linestyle': '-'},           # Gray solid
        'TD3': {'color': '#bcbd22', 'linestyle': '--'},          # Olive dashed
        'SAC': {'color': '#17becf', 'linestyle': '-'},           # Cyan solid
        'DJIA': {'color': '#000000', 'linestyle': '-'},          # Black solid
        # Add more methods here if needed
    }
    # Plot DJIA
    plt.plot(result.index, result['DJIA'], label="Dow Jones Index", linestyle=method_styles['DJIA']['linestyle'], color=method_styles['DJIA']['color'])

    # Define color palette and line styles
    color_palette = plt.get_cmap('tab10').colors  # Colorblind-friendly palette
    line_styles = ['-', '--', '-.', ':']  # Different line styles

    # Plot experiment groups
    for idx, (exp_name, stats) in enumerate(experiment_stats.items()):
        mean = stats['mean']
        std = stats['std']

        # Ensure mean and std are aligned with result.index
        mean = mean.reindex(result.index).fillna(method='ffill')
        std = std.reindex(result.index).fillna(0)

        # Assign colors and line styles
        # color = color_palette[idx % len(color_palette)]
        # linestyle = line_styles[idx % len(line_styles)]

        def exp_name_formatter(exp_name):
            exp_names = exp_name.split('_')
            if len(exp_names) == 1:
                return exp_name
            elif len(exp_names) == 2:
                return exp_names[1].upper()
            elif len(exp_names) == 3:
                return None
            elif len(exp_names) == 4:
                return exp_names[1].upper() + ' LoRA ' + 'GPT2'
            elif len(exp_names) == 6:
                return exp_names[1].upper() + ' LoRA ' + 'Random Weight ' + 'GPT2'
            else:
                return exp_name

        # Plot mean
        line, = plt.plot(result.index, mean, label=exp_name_formatter(exp_name), linestyle=method_styles[exp_name_formatter(exp_name)]['linestyle'], color=method_styles[exp_name_formatter(exp_name)]['color'])

        # Plot error bandsy (mean ± 1 std)
        plt.fill_between(result.index, mean - std, mean + std, color=method_styles[exp_name_formatter(exp_name)]['color'], alpha=0.2)

    # Set title and labels with enhanced formatting
    plt.title(f"Performance Comparison Under {current_algo.upper()} Expert Agent", fontsize=20, fontweight='bold')
    plt.xlabel("Date", fontsize=16, fontweight='bold')
    plt.ylabel("Total Asset Value ($)", fontsize=16, fontweight='bold')

    # import matplotlib.dates as mdates
    plt.xticks(result.index[0::30])
    # Add 'Test Phase' annotation with date range
    plt.text(0.5, 0.95, 'Test Phase: July 1, 2020 - October 29, 2021',
             transform=plt.gca().transAxes, fontsize=14, ha='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.5))

    # After all lines are plotted, sort the legend alphabetically
    handles, labels = plt.gca().get_legend_handles_labels()
    sorted_pairs = sorted(zip(labels, handles), key=lambda t: t[0].lower())  # Sort alphabetically, case-insensitive
    sorted_labels, sorted_handles = zip(*sorted_pairs)

    # Position legend outside the plot with sorted items
    plt.legend(sorted_handles, sorted_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=12)

    # Enhance layout and aesthetics
    plt.tight_layout()
    plt.grid(True, linestyle='--', alpha=0.3)

    # Save the plot with an informative filename
    plt.savefig(f'performance_comparison_DT-LoRA-GPT2_{current_algo.upper()}_Expert.png', dpi=300, bbox_inches='tight')
    plt.close()

    print(f"Results saved as 'performance_comparison_DT-LoRA-GPT2_{current_algo.upper()}_Expert.png'")
    print("----------------------------------------")

Running with A2C set to True
Stock Dimension: 29, State Space: 291


[*********************100%***********************]  1 of 1 completed

hit end!
Shape of DataFrame:  (319, 8)

=== Metrics Comparison for A2C ===
Method Cumulative Return (%)      MDD (%) Sharpe Ratio
   A2C            1.16 ± 0.0 -11.13 ± 0.0   0.23 ± 0.0


=== Rankings for A2C ===

Cumulative Return (%):
A2C: 1.16%

Maximum Drawdown (MDD %) [Lower is Better]:
A2C: -11.13%

Sharpe Ratio [Higher is Better]:
A2C: 0.23




Results saved as 'performance_comparison_DT-LoRA-GPT2_A2C_Expert.png'
----------------------------------------
Running with DDPG set to True
Stock Dimension: 29, State Space: 291


[*********************100%***********************]  1 of 1 completed

...

hit end!
Shape of DataFrame:  (319, 8)



=== Metrics Comparison for TD3 ===
Method Cumulative Return (%)     MDD (%) Sharpe Ratio
   TD3            4.96 ± 0.0 -9.88 ± 0.0   0.75 ± 0.0


=== Rankings for TD3 ===

Cumulative Return (%):
TD3: 4.96%

Maximum Drawdown (MDD %) [Lower is Better]:
TD3: -9.88%

Sharpe Ratio [Higher is Better]:
TD3: 0.75


Results saved as 'performance_comparison_DT-LoRA-GPT2_TD3_Expert.png'
----------------------------------------
Running with SAC set to True
Stock Dimension: 29, State Space: 291


[*********************100%***********************]  1 of 1 completed

hit end!
Shape of DataFrame:  (319, 8)

=== Metrics Comparison for SAC ===
Method Cumulative Return (%)     MDD (%) Sharpe Ratio
   SAC           24.66 ± 0.0 -8.28 ± 0.0   2.36 ± 0.0


=== Rankings for SAC ===

Cumulative Return (%):
SAC: 24.66%

Maximum Drawdown (MDD %) [Lower is Better]:
SAC: -8.28%

Sharpe Ratio [Higher is Better]:
SAC: 2.36




Results saved as 'performance_comparison_DT-LoRA-GPT2_SAC_Expert.png'
----------------------------------------


In [ ]:
# wandb.finish()

## Eval RL only

In [ ]:
#@title Init env

# Define environment parameters
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": config['initial_amount'],
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": config['cost_pct'],
    "sell_cost_pct": config['cost_pct'],
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

# Initialize trading environment
e_trade_gym = StockTradingEnv(
    df=trade,
    turbulence_threshold=70, risk_indicator_col='vix',
    **env_kwargs
)

env_trade, obs_trade = e_trade_gym.get_sb_env()

Stock Dimension: 29, State Space: 291


In [ ]:
#@title load models
# os.makedirs('./checkpoints', exist_ok=True)

algorithms = [
    'a2c',
    'ddpg',
    'ppo',
    'td3',
    'sac'
]

# Define a mapping for better legend labels
label_mapping = {
    # 'DT_LoRA_GPT2': 'DT-LoRA-GPT2',
    # 'DT_LoRA_Random_Weight_GPT2': 'DT-LoRA-Random-GPT2',
    # 'CQL': 'Conservative Q-Learning',
    # 'IQL': 'Implicit Q-Learning',
    # 'BC': 'Behavior Cloning',
    'A2C': 'A2C',
    'DDPG': 'DDPG',
    'PPO': 'PPO',
    'TD3': 'TD3',
    'SAC': 'SAC',
    'DJIA': 'Dow Jones Index'
}

for current_algo in algorithms:
    # Reset all algorithms to False
    if_using_a2c = False
    if_using_ddpg = False
    if_using_ppo = False
    if_using_td3 = False
    if_using_sac = False

    # Set the current algorithm to True
    if current_algo == 'a2c':
        if_using_a2c = True
    elif current_algo == 'ddpg':
        if_using_ddpg = True
    elif current_algo == 'ppo':
        if_using_ppo = True
    elif current_algo == 'td3':
        if_using_td3 = True
    elif current_algo == 'sac':
        if_using_sac = True

    # Reset algos_included for each iteration
    algos_included = ''

    print(f"Running with {current_algo.upper()} set to True")

# Load trained models
trained_a2c = A2C.load(os.path.join(TRAINED_MODEL_DIR, "agent_a2c")) if if_using_a2c else None
trained_ddpg = DDPG.load(os.path.join(TRAINED_MODEL_DIR, "agent_ddpg")) if if_using_ddpg else None
trained_ppo = PPO.load(os.path.join(TRAINED_MODEL_DIR, "agent_ppo")) if if_using_ppo else None
trained_td3 = TD3.load(os.path.join(TRAINED_MODEL_DIR, "agent_td3")) if if_using_td3 else None
trained_sac = SAC.load(os.path.join(TRAINED_MODEL_DIR, "agent_sac")) if if_using_sac else None

Running with A2C set to True
Running with DDPG set to True
Running with PPO set to True
Running with TD3 set to True
Running with SAC set to True


In [ ]:
#@title eval strategy

# Predict using trained models
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment=e_trade_gym) if if_using_a2c else (None, None)

df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg,
    environment=e_trade_gym) if if_using_ddpg else (None, None)

df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3,
    environment=e_trade_gym) if if_using_td3 else (None, None)

df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, environment=e_trade_gym) if if_using_sac else (None, None)

df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, environment=e_trade_gym) if if_using_ppo else (None, None)

# Set indices for result merging
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0]) if if_using_a2c else None
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0]) if if_using_ddpg else None
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0]) if if_using_ppo else None
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0]) if if_using_td3 else None
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0]) if if_using_sac else None

# Merge results
result = pd.DataFrame()
if if_using_a2c:
    algos_included += '_a2c'
    df_result_a2c.columns = ['A2C_' + col for col in df_result_a2c.columns]
    result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
if if_using_ddpg:
    algos_included += '_ddpg'
    df_result_ddpg.columns = ['DDPG_' + col for col in df_result_ddpg.columns]
    result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
if if_using_td3:
    algos_included += '_td3'
    df_result_td3.columns = ['TD3_' + col for col in df_result_td3.columns]
    result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
if if_using_sac:
    algos_included += '_sac'
    df_result_sac.columns = ['SAC_' + col for col in df_result_sac.columns]
    result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
if if_using_ppo:
    algos_included += '_ppo'
    df_result_ppo.columns = ['PPO_' + col for col in df_result_ppo.columns]
    result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)

# Function to load pickle files
def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# Get all directories under ./checkpoints/
checkpoint_dirs = [d for d in os.listdir('./checkpoints') if os.path.isdir(os.path.join('./checkpoints', d))]

# Process each checkpoint directory
for dir_name in checkpoint_dirs:
    dir_path = os.path.join('./checkpoints', dir_name)
    pkl_files = [f for f in os.listdir(dir_path) if f.startswith('total_asset_value_change_test') and f.endswith('.pkl')]

    for pkl_file in pkl_files:
        file_path = os.path.join(dir_path, pkl_file)
        data = load_pickle(file_path)
        data = data[:335]  # Limit data to first 335 points

        # Validate data length
        if len(data) != len(result.index):
            print(f"Warning: Data length mismatch for {pkl_file} in {dir_name}. Expected {len(result.index)}, got {len(data)}. Skipping this file.")
            continue

        # Use directory name and pkl file name (without extension) as column name
        column_name = f'{dir_name}_{os.path.splitext(pkl_file)[0]}'

        df = pd.DataFrame(data, columns=[column_name])
        df.index = result.index  # Assuming the dates align with the existing result dataframe
        result = pd.merge(result, df, how='outer', left_index=True, right_index=True)

# Create column names list with better formatting
col_name = []
if if_using_a2c: col_name.append('A2C')
if if_using_ddpg: col_name.append('DDPG')
if if_using_td3: col_name.append('TD3')
if if_using_sac: col_name.append('SAC')
if if_using_ppo: col_name.append('PPO')
col_name.extend(result.columns[len(col_name):])  # Add all other column names
result.columns = col_name

# Define test period
TEST_START_DATE = '2020-07-01'
TEST_END_DATE = '2021-10-29'

# Fetch DJIA data for the test period
df_dji = YahooDownloader(start_date=TEST_START_DATE,
                            end_date=TEST_END_DATE,
                            ticker_list=['dji']).fetch_data()

df_dji = df_dji[['date','close']]
fst_day = df_dji['close'].iloc[0]
dji = pd.DataFrame({
    'DJIA': df_dji['close'].div(fst_day).mul(1000000).values
}, index=df_dji['date'])

# Merge DJIA data using inner join to ensure alignment
result = pd.merge(result, dji, how='inner', left_index=True, right_index=True).fillna(method='bfill')

# Control variables
include_ensemble = False  # Set to False to exclude ensemble experiments
exclude_algo_experiments = False  # Set to True to exclude individual algo experiments based on if_using_... flags

if include_ensemble:
    algos_included += '_ensemble'

# Function to extract experiment name from directory name
def extract_experiment_name(dir_name):
    # This regex matches everything up to the last underscore and number
    match = re.match(r'(.+)_\d+$', dir_name)
    if match:
        return match.group(1)
    return dir_name

# Function to check if an experiment should be included
def should_include_experiment(exp_name):
    if not include_ensemble and 'ensemble' in exp_name.lower():
        return False
    if exclude_algo_experiments:
        algo_flags = {
            'a2c': if_using_a2c,
            'ddpg': if_using_ddpg,
            'ppo': if_using_ppo,
            'td3': if_using_td3,
            'sac': if_using_sac
        }
        for algo, flag in algo_flags.items():
            if algo in exp_name.lower() and not flag:
                return False
    return True

# Group similar experiments
experiment_groups = {}
for column in result.columns:
    if column not in ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO', 'DJIA']:
        exp_name = extract_experiment_name(column.split('_total_asset_value_change_test')[0])
        if should_include_experiment(exp_name):
            if exp_name not in experiment_groups:
                experiment_groups[exp_name] = []
            experiment_groups[exp_name].append(column)

print('experiment_groups', experiment_groups)

# Initialize a dictionary to store metrics for comparison
metrics_dict = {
    'Method': [],
    'Cumulative Return Mean (%)': [],
    'Cumulative Return Std (%)': [],
    'MDD Mean (%)': [],
    'MDD Std (%)': [],
    'Sharpe Ratio Mean': [],
    'Sharpe Ratio Std': []
}

# Calculate metrics for each experiment group
experiment_stats = {}
for exp_name, columns in experiment_groups.items():
    exp_data = result[columns].dropna()

    if exp_data.empty:
        print(f"Warning: No valid data for experiment '{exp_name}'. Skipping metrics calculation.")
        continue

    # Cumulative Return: (Final - Initial) / Initial * 100 for each run
    cumulative_returns = (exp_data.iloc[-1] - exp_data.iloc[0]) / exp_data.iloc[0] * 100

    # Handle potential division by zero or invalid calculations
    cumulative_returns = cumulative_returns.replace([np.inf, -np.inf], np.nan).dropna()

    if cumulative_returns.empty:
        print(f"Warning: No valid cumulative returns for experiment '{exp_name}'. Skipping metrics calculation.")
        continue

    cumulative_return_mean = cumulative_returns.mean()
    cumulative_return_std = cumulative_returns.std()

    # MDD: Calculate MDD for each run
    mdd_values = []
    for col in columns:
        asset_values = result[col].dropna()
        if asset_values.empty:
            continue
        mdd_run = calculate_mdd(asset_values)
        mdd_values.append(mdd_run)

    if not mdd_values:
        print(f"Warning: No valid MDD values for experiment '{exp_name}'. Skipping MDD calculation.")
        mdd_mean = np.nan
        mdd_std = np.nan
    else:
        mdd_mean = np.mean(mdd_values)
        mdd_std = np.std(mdd_values)

    # Sharpe Ratio: Calculate Sharpe for each run
    sharpe_ratios = []
    for col in columns:
        asset_values = result[col].dropna()
        if asset_values.empty:
            continue
        sharpe_run = calculate_sharpe_ratio(asset_values)
        sharpe_ratios.append(sharpe_run)

    if not sharpe_ratios:
        print(f"Warning: No valid Sharpe Ratios for experiment '{exp_name}'. Skipping Sharpe Ratio calculation.")
        sharpe_mean = np.nan
        sharpe_std = np.nan
    else:
        sharpe_mean = np.mean(sharpe_ratios)
        sharpe_std = np.std(sharpe_ratios)

    # Append to metrics_dict with mapped label
    mapped_exp_name = label_mapping.get(exp_name, exp_name)
    metrics_dict['Method'].append(mapped_exp_name)
    metrics_dict['Cumulative Return Mean (%)'].append(cumulative_return_mean)
    metrics_dict['Cumulative Return Std (%)'].append(cumulative_return_std)
    metrics_dict['MDD Mean (%)'].append(mdd_mean)
    metrics_dict['MDD Std (%)'].append(mdd_std)
    metrics_dict['Sharpe Ratio Mean'].append(sharpe_mean)
    metrics_dict['Sharpe Ratio Std'].append(sharpe_std)

    # Store in experiment_stats for plotting
    experiment_stats[mapped_exp_name] = {'mean': exp_data.mean(axis=1), 'std': exp_data.std(axis=1)}

# Calculate metrics for individual algorithms (A2C, DDPG, TD3, SAC, PPO)
individual_algos = ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO']
for algo in individual_algos:
    if algo in result.columns:
        # Check if this algorithm is already part of experiment_groups
        if label_mapping.get(algo, algo) in experiment_stats:
            print(f"Info: '{algo}' is already included in experiment groups. Skipping individual plotting to avoid duplication.")
            continue  # Skip to prevent duplicate plotting

        asset_values = result[algo].dropna()
        if asset_values.empty:
            print(f"Warning: No valid asset values for individual algorithm '{algo}'. Skipping metrics calculation.")
            continue
        # Cumulative Return
        cum_ret = (asset_values.iloc[-1] - asset_values.iloc[0]) / asset_values.iloc[0] * 100
        # Handle potential division by zero or invalid calculations
        if np.isinf(cum_ret) or np.isnan(cum_ret):
            cum_ret = np.nan
        # MDD
        mdd = calculate_mdd(asset_values)
        # Sharpe Ratio
        sharpe = calculate_sharpe_ratio(asset_values)
        # Append to metrics_dict with mapped label
        mapped_algo = label_mapping.get(algo, algo)
        metrics_dict['Method'].append(mapped_algo)
        metrics_dict['Cumulative Return Mean (%)'].append(cum_ret)
        metrics_dict['Cumulative Return Std (%)'].append(0.00)  # Single run, std is 0
        metrics_dict['MDD Mean (%)'].append(mdd)
        metrics_dict['MDD Std (%)'].append(0.00)  # Single run, std is 0
        metrics_dict['Sharpe Ratio Mean'].append(sharpe)
        metrics_dict['Sharpe Ratio Std'].append(0.00)  # Single run, std is 0

        # Store in experiment_stats for plotting
        experiment_stats[mapped_algo] = {'mean': asset_values, 'std': pd.Series([0]*len(asset_values), index=asset_values.index)}

# Convert metrics_dict to DataFrame
metrics_df = pd.DataFrame(metrics_dict)

# Drop any rows with NaN metrics to ensure clean tables
metrics_df = metrics_df.dropna(subset=['Cumulative Return Mean (%)', 'MDD Mean (%)', 'Sharpe Ratio Mean'])

# Create summary DataFrame with formatted strings
metrics_summary_df = metrics_df.copy()
metrics_summary_df['Cumulative Return (%)'] = metrics_df['Cumulative Return Mean (%)'].round(2).astype(str) + " ± " + metrics_df['Cumulative Return Std (%)'].round(2).astype(str)
metrics_summary_df['MDD (%)'] = metrics_df['MDD Mean (%)'].round(2).astype(str) + " ± " + metrics_df['MDD Std (%)'].round(2).astype(str)
metrics_summary_df['Sharpe Ratio'] = metrics_df['Sharpe Ratio Mean'].round(2).astype(str) + " ± " + metrics_df['Sharpe Ratio Std'].round(2).astype(str)
metrics_summary_df = metrics_summary_df[['Method', 'Cumulative Return (%)', 'MDD (%)', 'Sharpe Ratio']]

# Print the comparison table
print(f"\n=== Metrics Comparison for {current_algo.upper()} ===")
print(metrics_summary_df.to_string(index=False))
print("\n")

# Create separate DataFrames for ranking
ranking_cum_ret = metrics_df[['Method', 'Cumulative Return Mean (%)']].copy()
ranking_cum_ret = ranking_cum_ret.sort_values(by='Cumulative Return Mean (%)', ascending=False)

ranking_mdd = metrics_df[['Method', 'MDD Mean (%)']].copy()
ranking_mdd = ranking_mdd.sort_values(by='MDD Mean (%)', ascending=True)  # Lower MDD is better

ranking_sharpe = metrics_df[['Method', 'Sharpe Ratio Mean']].copy()
ranking_sharpe = ranking_sharpe.sort_values(by='Sharpe Ratio Mean', ascending=False)

# Print rankings
print(f"=== Rankings for {current_algo.upper()} ===")

print("\nCumulative Return (%):")
for idx, row in ranking_cum_ret.iterrows():
    print(f"{row['Method']}: {row['Cumulative Return Mean (%)']:.2f}%")

print("\nMaximum Drawdown (MDD %) [Lower is Better]:")
for idx, row in ranking_mdd.iterrows():
    print(f"{row['Method']}: {row['MDD Mean (%)']:.2f}%")

print("\nSharpe Ratio [Higher is Better]:")
for idx, row in ranking_sharpe.iterrows():
    print(f"{row['Method']}: {row['Sharpe Ratio Mean']:.2f}")

print("\n")

# Debugging: Check if all means align with result.index
for exp_name, stats in experiment_stats.items():
    mean_length = len(stats['mean'])
    result_length = len(result.index)
    if mean_length != result_length:
        print(f"Warning: Mean length for '{exp_name}' ({mean_length}) does not match result index length ({result_length}). Reindexing.")
        experiment_stats[exp_name]['mean'] = stats['mean'].reindex(result.index).fillna(method='ffill')
        experiment_stats[exp_name]['std'] = stats['std'].reindex(result.index).fillna(0)

# Plotting section
plt.figure(figsize=(16, 9))  # Increased figure size for better readability
method_styles = {
    'CQL': {'color': '#1f77b4', 'linestyle': '-'},           # Blue solid
    'IQL': {'color': '#ff7f0e', 'linestyle': '--'},          # Orange dashed
    'BC': {'color': '#2ca02c', 'linestyle': '-.'},           # Green dash-dot
    'DT LoRA GPT2': {'color': '#d62728', 'linestyle': ':'},  # Red dotted
    'DT LoRA Random Weight GPT2': {'color': '#9467bd', 'linestyle': '-'},  # Purple solid
    'A2C': {'color': '#8c564b', 'linestyle': '--'},          # Brown dashed
    'DDPG': {'color': '#e377c2', 'linestyle': '-'},          # Pink solid
    'PPO': {'color': '#7f7f7f', 'linestyle': '-'},           # Gray solid
    'TD3': {'color': '#bcbd22', 'linestyle': '--'},          # Olive dashed
    'SAC': {'color': '#17becf', 'linestyle': '-'},           # Cyan solid
    'DJIA': {'color': '#000000', 'linestyle': '-'},          # Black solid
    # Add more methods here if needed
}
# Plot DJIA
plt.plot(result.index, result['DJIA'], label="Dow Jones Index", linestyle=method_styles['DJIA']['linestyle'], color=method_styles['DJIA']['color'])

# Define color palette and line styles
color_palette = plt.get_cmap('tab10').colors  # Colorblind-friendly palette
line_styles = ['-', '--', '-.', ':']  # Different line styles

# Plot experiment groups
for idx, (exp_name, stats) in enumerate(experiment_stats.items()):
    mean = stats['mean']
    std = stats['std']

    # Ensure mean and std are aligned with result.index
    mean = mean.reindex(result.index).fillna(method='ffill')
    std = std.reindex(result.index).fillna(0)

    # Assign colors and line styles
    # color = color_palette[idx % len(color_palette)]
    # linestyle = line_styles[idx % len(line_styles)]

    def exp_name_formatter(exp_name):
        exp_names = exp_name.split('_')
        if len(exp_names) == 1:
            return exp_name
        elif len(exp_names) == 2:
            return exp_names[1].upper()
        elif len(exp_names) == 3:
            return None
        elif len(exp_names) == 4:
            return exp_names[1].upper() + ' LoRA ' + 'GPT2'
        elif len(exp_names) == 6:
            return exp_names[1].upper() + ' LoRA ' + 'Random Weight ' + 'GPT2'
        else:
            return exp_name

    # Plot mean
    line, = plt.plot(result.index, mean, label=exp_name_formatter(exp_name), linestyle=method_styles[exp_name_formatter(exp_name)]['linestyle'], color=method_styles[exp_name_formatter(exp_name)]['color'])

    # Plot error bandsy (mean ± 1 std)
    plt.fill_between(result.index, mean - std, mean + std, color=method_styles[exp_name_formatter(exp_name)]['color'], alpha=0.2)

# Set title and labels with enhanced formatting
plt.title(f"Performance Comparison Under {current_algo.upper()} Expert Agent", fontsize=20, fontweight='bold')
plt.xlabel("Date", fontsize=16, fontweight='bold')
plt.ylabel("Total Asset Value ($)", fontsize=16, fontweight='bold')

# import matplotlib.dates as mdates
plt.xticks(result.index[0::30])
# Add 'Test Phase' annotation with date range
plt.text(0.5, 0.95, 'Test Phase: July 1, 2020 - October 29, 2021',
            transform=plt.gca().transAxes, fontsize=14, ha='center',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.5))

# After all lines are plotted, sort the legend alphabetically
handles, labels = plt.gca().get_legend_handles_labels()
sorted_pairs = sorted(zip(labels, handles), key=lambda t: t[0].lower())  # Sort alphabetically, case-insensitive
sorted_labels, sorted_handles = zip(*sorted_pairs)

# Position legend outside the plot with sorted items
plt.legend(sorted_handles, sorted_labels, loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=12)

# Enhance layout and aesthetics
plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.3)

# Save the plot with an informative filename
plt.savefig(f'performance_comparison_DT-LoRA-GPT2_{current_algo.upper()}_Expert.png', dpi=300, bbox_inches='tight')
plt.close()

print(f"Results saved as 'performance_comparison_DT-LoRA-GPT2_{current_algo.upper()}_Expert.png'")
print("----------------------------------------")

hit end!
hit end!
hit end!
hit end!


[*********************100%***********************]  1 of 1 completed

hit end!
Shape of DataFrame:  (319, 8)
experiment_groups {}

=== Metrics Comparison for SAC ===
Method Cumulative Return (%)      MDD (%) Sharpe Ratio
   A2C           14.11 ± 0.0  -7.53 ± 0.0   1.89 ± 0.0
  DDPG           15.45 ± 0.0  -8.62 ± 0.0   1.95 ± 0.0
   TD3           14.66 ± 0.0 -11.75 ± 0.0   1.54 ± 0.0
   SAC           24.68 ± 0.0  -8.36 ± 0.0   2.36 ± 0.0
   PPO           14.92 ± 0.0  -9.13 ± 0.0    2.1 ± 0.0


=== Rankings for SAC ===

Cumulative Return (%):
SAC: 24.68%
DDPG: 15.45%
PPO: 14.92%
TD3: 14.66%
A2C: 14.11%

Maximum Drawdown (MDD %) [Lower is Better]:
TD3: -11.75%
PPO: -9.13%
DDPG: -8.62%
SAC: -8.36%
A2C: -7.53%

Sharpe Ratio [Higher is Better]:
SAC: 2.36
PPO: 2.10
DDPG: 1.95
A2C: 1.89
TD3: 1.54




Results saved as 'performance_comparison_DT-LoRA-GPT2_SAC_Expert.png'
----------------------------------------


## Eval RL-only (refactored)

In [ ]:
#@title Init env

# Define environment parameters
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": config['initial_amount'],
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": config['cost_pct'],
    "sell_cost_pct": config['cost_pct'],
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

# Initialize trading environment
e_trade_gym = StockTradingEnv(
    df=trade,
    turbulence_threshold=70, risk_indicator_col='vix',
    **env_kwargs
)

env_trade, obs_trade = e_trade_gym.get_sb_env()

Stock Dimension: 29, State Space: 291


In [ ]:
#@title load models

# Load variables from the config
if_using_a2c = config["if_using_a2c"]
if_using_ddpg = config["if_using_ddpg"]
if_using_ppo = config["if_using_ppo"]
if_using_td3 = config["if_using_td3"]
if_using_sac = config["if_using_sac"]

# Ensure at least one algorithm is enabled
if not any([if_using_a2c, if_using_ddpg, if_using_ppo, if_using_td3, if_using_sac]):
    raise ValueError("At least one algorithm must be set to True for the script to run.")

# Load trained models
trained_a2c = A2C.load(os.path.join(TRAINED_MODEL_DIR, "agent_a2c")) if if_using_a2c else None
trained_ddpg = DDPG.load(os.path.join(TRAINED_MODEL_DIR, "agent_ddpg")) if if_using_ddpg else None
trained_ppo = PPO.load(os.path.join(TRAINED_MODEL_DIR, "agent_ppo")) if if_using_ppo else None
trained_td3 = TD3.load(os.path.join(TRAINED_MODEL_DIR, "agent_td3")) if if_using_td3 else None
trained_sac = SAC.load(os.path.join(TRAINED_MODEL_DIR, "agent_sac")) if if_using_sac else None

In [ ]:
#@title get predictions

# Reset the result DataFrame
result = pd.DataFrame()

# Predict and store results for all enabled algorithms
for algo_name, trained_model, is_enabled in [
    ("A2C", trained_a2c, if_using_a2c),
    ("DDPG", trained_ddpg, if_using_ddpg),
    ("PPO", trained_ppo, if_using_ppo),
    ("TD3", trained_td3, if_using_td3),
    ("SAC", trained_sac, if_using_sac)
]:
    if is_enabled:
        df_account_value, df_actions = DRLAgent.DRL_prediction(
            model=trained_model, environment=e_trade_gym
        )
        df_result = df_account_value.set_index(df_account_value.columns[0])
        df_result.columns = [f"{algo_name}_{col}" for col in df_result.columns]
        result = pd.merge(result, df_result, how="outer", left_index=True, right_index=True)

hit end!
hit end!
hit end!
hit end!
hit end!


In [ ]:
#@title load DIJA for test period

# Define test period
TEST_START_DATE = trade['date'].iloc[0]
TEST_END_DATE = trade['date'].iloc[-1]

# Fetch DJIA data for the test period
df_dji = YahooDownloader(
    start_date=TEST_START_DATE,
    end_date=TEST_END_DATE,
    ticker_list=['dji']
).fetch_data()

# Merge DJIA data
df_dji = df_dji[['date','close']]
fst_day = df_dji['close'].iloc[0]
dji = pd.DataFrame({
    'DJIA': df_dji['close'].div(fst_day).mul(1000000).values
}, index=df_dji['date'])

# Merge DJIA data using inner join to ensure alignment
result = pd.merge(result, dji, how='inner', left_index=True, right_index=True).fillna(method='bfill')

# Create column names list with better formatting
col_name = []
if if_using_a2c: col_name.append('A2C')
if if_using_ddpg: col_name.append('DDPG')
if if_using_td3: col_name.append('TD3')
if if_using_sac: col_name.append('SAC')
if if_using_ppo: col_name.append('PPO')
col_name.extend(result.columns[len(col_name):])  # Add all other column names
result.columns = col_name

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (240, 8)


In [ ]:
#@title [DEPRECATED] filter experiments

# Control variables
include_ensemble = False  # Set to False to exclude ensemble experiments
exclude_algo_experiments = False  # Set to True to exclude individual algo experiments based on if_using_... flags

if include_ensemble:
    algos_included += '_ensemble'

# Function to extract experiment name from directory name
def extract_experiment_name(dir_name):
    # This regex matches everything up to the last underscore and number
    match = re.match(r'(.+)_\d+$', dir_name)
    if match:
        return match.group(1)
    return dir_name

# Function to check if an experiment should be included
def should_include_experiment(exp_name):
    if not include_ensemble and 'ensemble' in exp_name.lower():
        return False
    if exclude_algo_experiments:
        algo_flags = {
            'a2c': if_using_a2c,
            'ddpg': if_using_ddpg,
            'ppo': if_using_ppo,
            'td3': if_using_td3,
            'sac': if_using_sac
        }
        for algo, flag in algo_flags.items():
            if algo in exp_name.lower() and not flag:
                return False
    return True

# Group similar experiments
experiment_groups = {}
for column in result.columns:
    if column not in ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO', 'DJIA']:
        exp_name = extract_experiment_name(column.split('_total_asset_value_change_test')[0])
        if should_include_experiment(exp_name):
            if exp_name not in experiment_groups:
                experiment_groups[exp_name] = []
            experiment_groups[exp_name].append(column)

experiment_groups

{}

In [ ]:
#@title [DEPRECATED] Calculate metrics for each experiment group

# SHOULD USE ONE metrics_dict when running with groups (delete second initialization)
metrics_dict = {
    'Method': [],
    'Cumulative Return Mean (%)': [],
    'Cumulative Return Std (%)': [],
    'MDD Mean (%)': [],
    'MDD Std (%)': [],
    'Sharpe Ratio Mean': [],
    'Sharpe Ratio Std': []
}

# Calculate metrics for each experiment group
experiment_stats = {}
for exp_name, columns in experiment_groups.items():
    exp_data = result[columns].dropna()

    if exp_data.empty:
        print(f"Warning: No valid data for experiment '{exp_name}'. Skipping metrics calculation.")
        continue

    # Cumulative Return: (Final - Initial) / Initial * 100 for each run
    cumulative_returns = (exp_data.iloc[-1] - exp_data.iloc[0]) / exp_data.iloc[0] * 100

    # Handle potential division by zero or invalid calculations
    cumulative_returns = cumulative_returns.replace([np.inf, -np.inf], np.nan).dropna()

    if cumulative_returns.empty:
        print(f"Warning: No valid cumulative returns for experiment '{exp_name}'. Skipping metrics calculation.")
        continue

    cumulative_return_mean = cumulative_returns.mean()
    cumulative_return_std = cumulative_returns.std()

    # MDD: Calculate MDD for each run
    mdd_values = []
    for col in columns:
        asset_values = result[col].dropna()
        if asset_values.empty:
            continue
        mdd_run = calculate_mdd(asset_values)
        mdd_values.append(mdd_run)

    if not mdd_values:
        print(f"Warning: No valid MDD values for experiment '{exp_name}'. Skipping MDD calculation.")
        mdd_mean = np.nan
        mdd_std = np.nan
    else:
        mdd_mean = np.mean(mdd_values)
        mdd_std = np.std(mdd_values)

    # Sharpe Ratio: Calculate Sharpe for each run
    sharpe_ratios = []
    for col in columns:
        asset_values = result[col].dropna()
        if asset_values.empty:
            continue
        sharpe_run = calculate_sharpe_ratio(asset_values)
        sharpe_ratios.append(sharpe_run)

    if not sharpe_ratios:
        print(f"Warning: No valid Sharpe Ratios for experiment '{exp_name}'. Skipping Sharpe Ratio calculation.")
        sharpe_mean = np.nan
        sharpe_std = np.nan
    else:
        sharpe_mean = np.mean(sharpe_ratios)
        sharpe_std = np.std(sharpe_ratios)

    # Append to metrics_dict with mapped label
    mapped_exp_name = label_mapping.get(exp_name, exp_name)
    metrics_dict['Method'].append(mapped_exp_name)
    metrics_dict['Cumulative Return Mean (%)'].append(cumulative_return_mean)
    metrics_dict['Cumulative Return Std (%)'].append(cumulative_return_std)
    metrics_dict['MDD Mean (%)'].append(mdd_mean)
    metrics_dict['MDD Std (%)'].append(mdd_std)
    metrics_dict['Sharpe Ratio Mean'].append(sharpe_mean)
    metrics_dict['Sharpe Ratio Std'].append(sharpe_std)

    # Store in experiment_stats for plotting
    experiment_stats[mapped_exp_name] = {'mean': exp_data.mean(axis=1), 'std': exp_data.std(axis=1)}

In [ ]:
#@title Calculate metrics for individual algorithms

metrics_dict = {
    'Method': [],
    'Cumulative Return Mean (%)': [],
    'Cumulative Return Std (%)': [],
    'MDD Mean (%)': [],
    'MDD Std (%)': [],
    'Sharpe Ratio Mean': [],
    'Sharpe Ratio Std': []
}

result.index = pd.to_datetime(result.index)
individual_algos = ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO']
for algo in individual_algos:
    if algo in result.columns:
        # # Check if this algorithm is already part of experiment_groups
        # if label_mapping.get(algo, algo) in experiment_stats:
        #     print(f"Info: '{algo}' is already included in experiment groups. Skipping individual plotting to avoid duplication.")
        #     continue  # Skip to prevent duplicate plotting

        asset_values = result[algo].dropna()
        if asset_values.empty:
            print(f"Warning: No valid asset values for individual algorithm '{algo}'. Skipping metrics calculation.")
            continue
        # Cumulative Return
        cum_ret = (asset_values.iloc[-1] - asset_values.iloc[0]) / asset_values.iloc[0] * 100
        # Handle potential division by zero or invalid calculations
        if np.isinf(cum_ret) or np.isnan(cum_ret):
            cum_ret = np.nan
        # MDD
        mdd = calculate_mdd(asset_values)
        # Sharpe Ratio
        sharpe = calculate_sharpe_ratio(asset_values)
        # Append to metrics_dict with mapped label
        mapped_algo = label_mapping.get(algo, algo)
        metrics_dict['Method'].append(mapped_algo)
        metrics_dict['Cumulative Return Mean (%)'].append(cum_ret)
        metrics_dict['Cumulative Return Std (%)'].append(0.00)  # Single run, std is 0
        metrics_dict['MDD Mean (%)'].append(mdd)
        metrics_dict['MDD Std (%)'].append(0.00)  # Single run, std is 0
        metrics_dict['Sharpe Ratio Mean'].append(sharpe)
        metrics_dict['Sharpe Ratio Std'].append(0.00)  # Single run, std is 0

        # Store in experiment_stats for plotting
        experiment_stats[mapped_algo] = {'mean': asset_values, 'std': pd.Series([0]*len(asset_values), index=asset_values.index)}

# Convert metrics_dict to DataFrame
metrics_df = pd.DataFrame(metrics_dict)

# Drop any rows with NaN metrics to ensure clean tables
metrics_df = metrics_df.dropna(subset=['Cumulative Return Mean (%)', 'MDD Mean (%)', 'Sharpe Ratio Mean'])

# Create summary DataFrame with formatted strings
metrics_summary_df = metrics_df.copy()
metrics_summary_df['Cumulative Return (%)'] = metrics_df['Cumulative Return Mean (%)'].round(2).astype(str) + " ± " + metrics_df['Cumulative Return Std (%)'].round(2).astype(str)
metrics_summary_df['MDD (%)'] = metrics_df['MDD Mean (%)'].round(2).astype(str) + " ± " + metrics_df['MDD Std (%)'].round(2).astype(str)
metrics_summary_df['Sharpe Ratio'] = metrics_df['Sharpe Ratio Mean'].round(2).astype(str) + " ± " + metrics_df['Sharpe Ratio Std'].round(2).astype(str)
metrics_summary_df = metrics_summary_df[['Method', 'Cumulative Return (%)', 'MDD (%)', 'Sharpe Ratio']]

In [ ]:
#@title Print the comparison table

print(f"\n=== Metrics Comparison ===")
print(metrics_summary_df.to_string(index=False))
print("\n")

# Create separate DataFrames for ranking
ranking_cum_ret = metrics_df[['Method', 'Cumulative Return Mean (%)']].copy()
ranking_cum_ret = ranking_cum_ret.sort_values(by='Cumulative Return Mean (%)', ascending=False)

ranking_mdd = metrics_df[['Method', 'MDD Mean (%)']].copy()
ranking_mdd = ranking_mdd.sort_values(by='MDD Mean (%)', ascending=True)  # Lower MDD is better

ranking_sharpe = metrics_df[['Method', 'Sharpe Ratio Mean']].copy()
ranking_sharpe = ranking_sharpe.sort_values(by='Sharpe Ratio Mean', ascending=False)

# Print rankings
print(f"=== Rankings ===")

print("\nCumulative Return (%):")
for idx, row in ranking_cum_ret.iterrows():
    print(f"{row['Method']}: {row['Cumulative Return Mean (%)']:.2f}%")

print("\nMaximum Drawdown (MDD %) [Lower is Better]:")
for idx, row in ranking_mdd.iterrows():
    print(f"{row['Method']}: {row['MDD Mean (%)']:.2f}%")

print("\nSharpe Ratio [Higher is Better]:")
for idx, row in ranking_sharpe.iterrows():
    print(f"{row['Method']}: {row['Sharpe Ratio Mean']:.2f}")

print("\n")

# Debugging: Check if all means align with result.index
for exp_name, stats in experiment_stats.items():
    mean_length = len(stats['mean'])
    result_length = len(result.index)
    if mean_length != result_length:
        print(f"Warning: Mean length for '{exp_name}' ({mean_length}) does not match result index length ({result_length}). Reindexing.")
        experiment_stats[exp_name]['mean'] = stats['mean'].reindex(result.index).fillna(method='ffill')
        experiment_stats[exp_name]['std'] = stats['std'].reindex(result.index).fillna(0)


=== Metrics Comparison ===
Method Cumulative Return (%)      MDD (%) Sharpe Ratio
   A2C           11.16 ± 0.0 -19.65 ± 0.0   0.52 ± 0.0
  DDPG             3.5 ± 0.0 -23.02 ± 0.0   0.27 ± 0.0
   TD3           16.08 ± 0.0 -19.77 ± 0.0   0.72 ± 0.0
   SAC            2.82 ± 0.0 -22.32 ± 0.0   0.24 ± 0.0
   PPO           13.18 ± 0.0 -33.54 ± 0.0   0.54 ± 0.0


=== Rankings ===

Cumulative Return (%):
TD3: 16.08%
PPO: 13.18%
A2C: 11.16%
DDPG: 3.50%
SAC: 2.82%

Maximum Drawdown (MDD %) [Lower is Better]:
PPO: -33.54%
DDPG: -23.02%
SAC: -22.32%
TD3: -19.77%
A2C: -19.65%

Sharpe Ratio [Higher is Better]:
TD3: 0.72
PPO: 0.54
A2C: 0.52
DDPG: 0.27
SAC: 0.24




In [215]:
#@title Calculate metrics for individual algorithms (w/annualized returns)

def calculate_metrics(result):
    metrics_dict = {
        'Method': [],
        'Cumulative Return Mean (%)': [],
        # 'Cumulative Return Std (%)': [],
        'Annualized Return Mean (%)': [],
        # 'Annualized Return Std (%)': [],
        'MDD Mean (%)': [],
        'MDD Std (%)': [],
        'Sharpe Ratio Mean': [],
        'Sharpe Ratio Std': []
    }

    individual_algos = ['A2C', 'DDPG', 'TD3', 'SAC', 'PPO']
    for algo in individual_algos:
        if algo in result.columns:
            # # Check if this algorithm is already part of experiment_groups
            # if label_mapping.get(algo, algo) in experiment_stats:
            #     print(f"Info: '{algo}' is already included in experiment groups. Skipping individual plotting to avoid duplication.")
            #     continue  # Skip to prevent duplicate plotting

            asset_values = result[algo].dropna()
            if asset_values.empty:
                print(f"Warning: No valid asset values for individual algorithm '{algo}'. Skipping metrics calculation.")
                continue

            cum_ret = (asset_values.iloc[-1] - asset_values.iloc[0]) / asset_values.iloc[0] * 100

            # Handle potential division by zero or invalid calculations
            if np.isinf(cum_ret) or np.isnan(cum_ret):
                cum_ret = np.nan

            # Calculate annualized return
            num_days = (asset_values.index[-1] - asset_values.index[0]).days
            ann_ret = ( (1 + cum_ret / 100) ** (365 / num_days) - 1 ) * 100

            # MDD
            mdd = calculate_mdd(asset_values)
            # Sharpe Ratio
            sharpe = calculate_sharpe_ratio(asset_values)
            # Append to metrics_dict with mapped label
            mapped_algo = label_mapping.get(algo, algo)
            metrics_dict['Method'].append(mapped_algo)
            metrics_dict['Cumulative Return Mean (%)'].append(cum_ret)
            # metrics_dict['Cumulative Return Std (%)'].append(0.00)  # Single run, std is 0
            metrics_dict['Annualized Return Mean (%)'].append(ann_ret)
            # metrics_dict['Annualized Return Std (%)'].append(0.00)  # Single run, std is 0
            metrics_dict['MDD Mean (%)'].append(mdd)
            metrics_dict['MDD Std (%)'].append(0.00)  # Single run, std is 0
            metrics_dict['Sharpe Ratio Mean'].append(sharpe)
            metrics_dict['Sharpe Ratio Std'].append(0.00)  # Single run, std is 0

            # Store in experiment_stats for plotting
            experiment_stats[mapped_algo] = {'mean': asset_values, 'std': pd.Series([0]*len(asset_values), index=asset_values.index)}

    # Convert metrics_dict to DataFrame
    metrics_df = pd.DataFrame(metrics_dict)

    # Drop any rows with NaN metrics to ensure clean tables
    metrics_df = metrics_df.dropna(subset=['Cumulative Return Mean (%)', 'Annualized Return Mean (%)', 'MDD Mean (%)', 'Sharpe Ratio Mean'])

    # Create summary DataFrame with formatted strings
    metrics_summary_df = metrics_df.copy()
    metrics_summary_df['Cumulative Return (%)'] = metrics_df['Cumulative Return Mean (%)'].round(2).astype(str) \
        #  + " ± " + metrics_df['Cumulative Return Std (%)'].round(2).astype(str)
    metrics_summary_df['Annualized Return (%)'] = metrics_df['Annualized Return Mean (%)'].round(2).astype(str) \
        #  + " ± " + metrics_df['Annualized Return Std (%)'].round(2).astype(str)
    metrics_summary_df['MDD (%)'] = metrics_df['MDD Mean (%)'].round(2).astype(str) \
        #  + " ± " + metrics_df['MDD Std (%)'].round(2).astype(str)
    metrics_summary_df['Sharpe Ratio'] = metrics_df['Sharpe Ratio Mean'].round(2).astype(str) \
        #  + " ± " + metrics_df['Sharpe Ratio Std'].round(2).astype(str)
    metrics_summary_df = metrics_summary_df[['Method', 'Cumulative Return (%)', 'Annualized Return (%)', 'MDD (%)', 'Sharpe Ratio']]

    return metrics_summary_df

metrics_summary_df = calculate_metrics(result)
metrics_summary_df

,Method,Cumulative Return (%),Annualized Return (%),MDD (%),Sharpe Ratio
0,A2C,11.16,11.29,-19.65,0.52
1,DDPG,3.5,3.54,-23.02,0.27
2,TD3,16.08,16.27,-19.77,0.72
3,SAC,2.82,2.85,-22.32,0.24
4,PPO,13.18,13.34,-33.54,0.54


In [216]:
#@title Print the comparison table (w/annualized returns)

print(f"\n=== Metrics Comparison ===")
print(metrics_summary_df.to_string(index=False))
print("\n")

# Create separate DataFrames for rankings
ranking_cum_ret = metrics_df[['Method', 'Cumulative Return Mean (%)']].copy()
ranking_cum_ret = ranking_cum_ret.sort_values(by='Cumulative Return Mean (%)', ascending=False)

ranking_annualized_ret = metrics_df[['Method', 'Annualized Return Mean (%)']].copy()
ranking_annualized_ret = ranking_annualized_ret.sort_values(by='Annualized Return Mean (%)', ascending=False)

ranking_mdd = metrics_df[['Method', 'MDD Mean (%)']].copy()
ranking_mdd = ranking_mdd.sort_values(by='MDD Mean (%)', ascending=False)  # Lower abs(MDD) is better

ranking_sharpe = metrics_df[['Method', 'Sharpe Ratio Mean']].copy()
ranking_sharpe = ranking_sharpe.sort_values(by='Sharpe Ratio Mean', ascending=False)

# Print rankings
print(f"=== Rankings ===")

print("\nCumulative Return (%):")
for idx, row in ranking_cum_ret.iterrows():
    print(f"{row['Method']}: {row['Cumulative Return Mean (%)']:.2f}%")

print("\nAnnualized Return (%):")
for idx, row in ranking_annualized_ret.iterrows():
    print(f"{row['Method']}: {row['Annualized Return Mean (%)']:.2f}%")

print("\nMaximum Drawdown (MDD %) [Lower absolute values is Better]:")
for idx, row in ranking_mdd.iterrows():
    print(f"{row['Method']}: {row['MDD Mean (%)']:.2f}%")

print("\nSharpe Ratio [Higher is Better]:")
for idx, row in ranking_sharpe.iterrows():
    print(f"{row['Method']}: {row['Sharpe Ratio Mean']:.2f}")

print("\n")

# Debugging: Check if all means align with result.index
for exp_name, stats in experiment_stats.items():
    mean_length = len(stats['mean'])
    result_length = len(result.index)
    if mean_length != result_length:
        print(f"Warning: Mean length for '{exp_name}' ({mean_length}) does not match result index length ({result_length}). Reindexing.")
        experiment_stats[exp_name]['mean'] = stats['mean'].reindex(result.index).fillna(method='ffill')
        experiment_stats[exp_name]['std'] = stats['std'].reindex(result.index).fillna(0)


=== Metrics Comparison ===
Method Cumulative Return (%) Annualized Return (%) MDD (%) Sharpe Ratio
   A2C                 11.16                 11.29  -19.65         0.52
  DDPG                   3.5                  3.54  -23.02         0.27
   TD3                 16.08                 16.27  -19.77         0.72
   SAC                  2.82                  2.85  -22.32         0.24
   PPO                 13.18                 13.34  -33.54         0.54


=== Rankings ===

Cumulative Return (%):
TD3: 16.08%
PPO: 13.18%
A2C: 11.16%
DDPG: 3.50%
SAC: 2.82%

Annualized Return (%):
TD3: 16.27%
PPO: 13.34%
A2C: 11.29%
DDPG: 3.54%
SAC: 2.85%

Maximum Drawdown (MDD %) [Lower absolute values is Better]:
A2C: -19.65%
TD3: -19.77%
SAC: -22.32%
DDPG: -23.02%
PPO: -33.54%

Sharpe Ratio [Higher is Better]:
TD3: 0.72
PPO: 0.54
A2C: 0.52
DDPG: 0.27
SAC: 0.24




In [224]:
#@title log_test_metrics
def log_test_metrics():
    # Calculate maximum values for each metric
    max_cum_return = metrics_summary_df['Cumulative Return (%)'].max()
    max_ann_return = metrics_summary_df['Annualized Return (%)'].max()
    max_mdd = metrics_summary_df['MDD (%)'].max()
    max_sharpe_ratio = metrics_summary_df['Sharpe Ratio'].max()

    # Log metrics for each model and maximum values
    for _, row in metrics_summary_df.iterrows():
        model_name = row['Method'].lower()  # Convert the model name to lowercase

        # Log per-model metrics
        wandb.run.log({
            'test': {
                f'{model_name}/cum_return': row['Cumulative Return (%)'],
                f'{model_name}/ann_return': row['Annualized Return (%)'],
                f'{model_name}/mdd': row['MDD (%)'],
                f'{model_name}/sharpe_ratio': row['Sharpe Ratio']
            }
        })

    # Log maximum values across models
    wandb.run.log({
        'test': {
            'max_cum_return': max_cum_return,
            'max_ann_return': max_ann_return,
            'max_mdd': max_mdd,
            'max_sharpe_ratio': max_sharpe_ratio
        }
    })

In [ ]:
#@title plot results

# Plotting section
plt.figure(figsize=(16, 9))  # Increased figure size for better readability
method_styles = {
    'CQL': {'color': '#1f77b4', 'linestyle': '-'},           # Blue solid
    'IQL': {'color': '#ff7f0e', 'linestyle': '--'},          # Orange dashed
    'BC': {'color': '#2ca02c', 'linestyle': '-.'},           # Green dash-dot
    'DT LoRA GPT2': {'color': '#d62728', 'linestyle': ':'},  # Red dotted
    'DT LoRA Random Weight GPT2': {'color': '#9467bd', 'linestyle': '-'},  # Purple solid
    'A2C': {'color': '#8c564b', 'linestyle': '--'},          # Brown dashed
    'DDPG': {'color': '#e377c2', 'linestyle': '-'},          # Pink solid
    'PPO': {'color': '#7f7f7f', 'linestyle': '-'},           # Gray solid
    'TD3': {'color': '#bcbd22', 'linestyle': '--'},          # Olive dashed
    'SAC': {'color': '#17becf', 'linestyle': '-'},           # Cyan solid
    'DJIA': {'color': '#000000', 'linestyle': '-'},          # Black solid
    # Add more methods here if needed
}
# Plot DJIA
plt.plot(result.index, result['DJIA'], label="Dow Jones Index", linestyle=method_styles['DJIA']['linestyle'], color=method_styles['DJIA']['color'])

# Define color palette and line styles
color_palette = plt.get_cmap('tab10').colors  # Colorblind-friendly palette
line_styles = ['-', '--', '-.', ':']  # Different line styles

# Plot experiment groups
for idx, (exp_name, stats) in enumerate(experiment_stats.items()):
    mean = stats['mean']
    std = stats['std']

    # Ensure mean and std are aligned with result.index
    mean = mean.reindex(result.index).fillna(method='ffill')
    std = std.reindex(result.index).fillna(0)

    # Assign colors and line styles
    # color = color_palette[idx % len(color_palette)]
    # linestyle = line_styles[idx % len(line_styles)]

    def exp_name_formatter(exp_name):
        exp_names = exp_name.split('_')
        if len(exp_names) == 1:
            return exp_name
        elif len(exp_names) == 2:
            return exp_names[1].upper()
        elif len(exp_names) == 3:
            return None
        elif len(exp_names) == 4:
            return exp_names[1].upper() + ' LoRA ' + 'GPT2'
        elif len(exp_names) == 6:
            return exp_names[1].upper() + ' LoRA ' + 'Random Weight ' + 'GPT2'
        else:
            return exp_name

    # Plot mean
    line, = plt.plot(result.index, mean, label=exp_name_formatter(exp_name), linestyle=method_styles[exp_name_formatter(exp_name)]['linestyle'], color=method_styles[exp_name_formatter(exp_name)]['color'])

    # Plot error bandsy (mean ± 1 std)
    plt.fill_between(result.index, mean - std, mean + std, color=method_styles[exp_name_formatter(exp_name)]['color'], alpha=0.2)